## Preprocess your data using XCMS3 and export data files for feature-based molecular networking through GNPS

To follow this example tutorial, download the folder named <tt>'peak/AMG_Plant_subset'</tt> from: <br>
https://gnps.ucsd.edu/ProteoSAFe/result.jsp?task=d74ca92d9dec4e2883f28506c670e3ca&view=advanced_view

load libraries and GNPS export source function



In [ ]:
source('customFunctions.R')
library(RColorBrewer)
library(xcms)


Use socket based parallel processing on Windows systems



In [ ]:
if (.Platform$OS.type == "unix") {
  register(bpstart(MulticoreParam(4)))
} else {
  register(bpstart(SnowParam(4)))
}


### Load data

load all .mzXML files and define sample grouping



In [ ]:
mzXMLfiles <- paste0('MSV000079204/',list.files(path = 'MSV000079204/', pattern = ".mzXML$", recursive = TRUE))
s_groups <- rep(dir('MSV000079204/'), each=2)
pheno <- data.frame(sample_name = basename(mzXMLfiles),
                    sample_group = s_groups, stringsAsFactors = FALSE)

In [ ]:
head(pheno)


read all raw data, including MS2 level



In [ ]:
rawData <- readMSData(mzXMLfiles, centroided. = TRUE, pdata = new("NAnnotatedDataFrame", pheno), mode = "onDisk")


create a base peak chromatogram (BPC) of your data for visual inspection



In [ ]:
group_colors <- paste0(brewer.pal(3, "Set1")[1:2], "60")
names(group_colors) <- unique(pheno$sample_group)
bpis <- chromatogram(rawData, aggregationFun = "max")
plot(bpis, col = group_colors[rawData$sample_group])


### Peak picking

define settings for the centWave peak detection.



In [ ]:
cwp <- CentWaveParam(snthresh = 3, noise = 5000, peakwidth = c(5,30), ppm = 10)


perform peak picking



In [ ]:
processedData <- findChromPeaks(rawData, param = cwp)


get an overview of the detected peaks, using a heatmap



In [ ]:
plotChromPeakImage(processedData, binSize = 10)


Plot selected chromatogram:


In [ ]:
## Define the rt and m/z range of the peak area
rtr <- c(2000, 3000)
mzr <- c(0, 1000)
## extract the chromatogram
chr_raw <- chromatogram(rawData, mz = mzr, rt = rtr, aggregationFun='max')
plot(chr_raw, col = group_colors[chr_raw$sample_group], lwd = 2)




### Retention time alignment

do a obiwarp alignment using the default settings



In [ ]:
processedData <- adjustRtime(processedData, param = ObiwarpParam())


plot the difference of adjusted to raw retention time



In [ ]:
plotAdjustedRtime(processedData)


### Peak grouping

define the PeakDensityParam



In [ ]:
pdp <- PeakDensityParam(sampleGroups = processedData$sample_group, minFraction = 0.10)
processedData <- groupChromPeaks(processedData, param = pdp)


### Gap filling

fill in missing peaks



In [ ]:
processed_Data <- fillChromPeaks(processedData)


### Export data

#### export MS1 and MS2 features

export .mgf file <br>
all features, independent of whether they contain MS2 data or MS1 only will be saved to <tt>'ms2spectra_all.mgf'</tt>. This file can for example be used to do <i>in silico</i> structure prediction through [SIRIUS+CSI:FingerID](https://bio.informatik.uni-jena.de/software/sirius/):



In [ ]:
## export the individual spectra into a .mgf file
filteredMs2Spectra <- featureSpectra(processedData, return.type = "Spectra")
filteredMs2Spectra@listData <- lapply(filteredMs2Spectra, clean, all = TRUE)
filteredMs2Spectra <- formatSpectraForGNPS(filteredMs2Spectra)

In [ ]:
writeMgfData(filteredMs2Spectra, "ms2spectra_all.mgf")


export peak area quantification table <br>
a peak area quantification table contains features and respective per sample peak areas in columns. The file is saved to <tt>'xcms_all.txt'</tt>.



In [ ]:
## get data
featuresDef <- featureDefinitions(processedData)
featuresIntensities <- featureValues(processedData, value = "into")

## generate data table
dataTable <- merge(featuresDef, featuresIntensities, by=0, all=TRUE)
dataTable <- dataTable[,!(names(dataTable) %in% c("peakidx"))]

In [ ]:
head(dataTable)

In [ ]:
write.table(dataTable, "xcms_all.txt", sep = "\t", quote = FALSE, row.names = FALSE)


#### export MS2 features only

export .mgf file <br>
This file can be submitted to GNPS [feature-based molecular networking](https://ccms-ucsd.github.io/GNPSDocumentation/featurebasedmolecularnetworking/):



In [ ]:
## Select for each feature the Spectrum2 with the largest TIC.
filteredMs2spectra_maxTic <- endoapply(split(filteredMs2Spectra, mcols(filteredMs2Spectra)$feature_id), function(z) z[which.max(lapply(intensity(z), sum))])
filteredMs2spectra_maxTic <- unlist(filteredMs2spectra_maxTic)

In [ ]:
writeMgfData(filteredMs2spectra_maxTic, "ms2spectra_maxTic.mgf")


export peak area quantification table <br>
the peak area quantification table contains features and respective per sample peak areas in columns. The file is saved to <tt>'xcms_onlyMS2.txt'</tt>. This file can be submitted to GNPS [feature-based molecular networking](https://ccms-ucsd.github.io/GNPSDocumentation/featurebasedmolecularnetworking/):



In [ ]:
## filter data table to contain only peaks with MSMS DF[ , !(names(DF) %in% drops)]
filteredDataTable <- dataTable[which(dataTable$Row.names %in% filteredMs2Spectra@elementMetadata$feature_id),]

In [ ]:
head(filteredDataTable)

In [ ]:
write.table(filteredDataTable, "xcms_onlyMS2.txt", sep = "\t", quote = FALSE, row.names = FALSE)